<a href="https://colab.research.google.com/github/kalmuroth/python-exo/blob/master/Analyse_Donn%C3%A9e_WOW_2008_LB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Nous allons étudier des données qui correspondent aux connexion de personnages-joueurs dans le jeu-vidéo World Of Warcraft sur un serveur en 2008.

Chaque row dans le fichier .csv correspond à une connexion quotidienne (snapshot) d'un joueur.

Sur un axe de 1 ans (année 2008), nous avons un total de 10 millions de connexion.

Je vais utiliser les librairies Python Pandas/Numpy/Seaborn pour étudier et analyser ces données visuellement pour démontrer les différents impact de l'extension sur le comportement des joueurs.

In [10]:
import numpy as np
import pandas as pd
import seaborn as sns
import os
import gc
from datetime import timedelta
import random
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
!ls


Mounted at /content/drive
drive  sample_data


# Préparer les données <a id="1"></a>

### Fichier en entrée <a id="2"></a>

In [2]:
print('%-33s %d' % ('Nombre de Fichier .csv :', len(os.listdir('./drive/MyDrive/input'))))
for i in range(34):
    print('-',end='')
print('-')
for file in os.listdir("./drive/MyDrive/input"):
    unit = 'MB'
    size = os.stat('./drive/MyDrive/input/' + file).st_size
    if round(size / 2**20, 2) < 0.5:
        size = round(size / 2**10, 2)
        unit = 'KB'
    else:
        size = round(size / 2**20, 2)
    print('%-25s %6.2f %2s' % (file, size, unit))

Nombre de Fichier .csv :          1
-----------------------------------
wowah_data.csv            623.93 MB


# Présentation des données <a id="1"></a>

### Les Joueurs <a id="2"></a>

On va commencer par quelque chose de simple, regarder un peu ce que représente un row dans nos données (on parse le dataframe).

In [9]:
wowah_data = pd.read_csv('./drive/MyDrive/input/wowah_data.csv', sep = ',', skipinitialspace=True);
wowah_data['timestamp'] = pd.to_datetime(wowah_data['timestamp'])
wowah_data['dates'] = wowah_data['timestamp'].dt.date
wowah_data.loc[wowah_data['timestamp'] >= '2008-11-18','extention'] = 'Extention 1'
wowah_data.loc[wowah_data['timestamp'] < '2008-11-18', 'extention'] = 'Extention 2'

dict_color = {'Death Knight': '#C41F3B',
                'Shaman': '#0070DE',
                'Druid': '#FF7D0A',
                'Rogue': '#FFF569',
                'Priest': '#FFFFFF',
                'Paladin': '#F58CBA',
                'Warrior': '#C79C6E',
                'Warlock': '#8787ED',
                'Mage': '#40C7EB',
                'Hunter': '#A9D271'}

wowah_data['Class_color']  = wowah_data.charclass.map(dict_color)

wowah_data['Date'] =  pd.to_datetime(wowah_data['timestamp'], format='%Y-%m-%d')
wowah_data["Day_of_Week"] = wowah_data.Date.dt.weekday
wowah_data["First_day_of_the_week"] = wowah_data.Date - wowah_data.Day_of_Week * timedelta(days=1)
wowah_data.drop(['Day_of_Week', 'Date'], axis = 1, inplace = True)
wowah_data["First_day_of_the_week"] = wowah_data["First_day_of_the_week"].dt.date

col = {}
for i in wowah_data.zone.unique() :
    color = "%06x" % random.randint(0, 0xFFFFFF)
    col[i] = '#' + color 
wowah_data['color_zone'] = wowah_data['zone'].map(col)

wowah_data.head()

,char,level,race,charclass,zone,guild,timestamp,dates,extention,Class_color,First_day_of_the_week,color_zone
0,59425,1,Orc,Rogue,Orgrimmar,165,2008-01-01 00:02:04,2008-01-01,Extention 2,#FFF569,2007-12-31,#b1d2ae
1,65494,9,Orc,Hunter,Durotar,-1,2008-01-01 00:02:04,2008-01-01,Extention 2,#A9D271,2007-12-31,#e078dc
2,65325,14,Orc,Warrior,Ghostlands,-1,2008-01-01 00:02:04,2008-01-01,Extention 2,#C79C6E,2007-12-31,#b19858
3,65490,18,Orc,Hunter,Ghostlands,-1,2008-01-01 00:02:04,2008-01-01,Extention 2,#A9D271,2007-12-31,#b19858
4,2288,60,Orc,Hunter,Hellfire Peninsula,-1,2008-01-01 00:02:09,2008-01-01,Extention 2,#A9D271,2007-12-31,#87cf06


Chaque ligne représente la connexion d'un joueur sur son personnage. 
Seulement la première connexion quotidienne a été enregistrer.

Timestamp représente le moment où le joueur s'est connecté la première fois dans la journée.

In [4]:
wowah_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10826734 entries, 0 to 10826733
Data columns (total 12 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   char                   int64         
 1   level                  int64         
 2   race                   object        
 3   charclass              object        
 4   zone                   object        
 5   guild                  int64         
 6   timestamp              datetime64[ns]
 7   dates                  object        
 8   extention              object        
 9   Class_color            object        
 10  First_day_of_the_week  object        
 11  color_zone             object        
dtypes: datetime64[ns](1), int64(3), object(8)
memory usage: 991.2+ MB


Comme une snapshot n'est pas représentatif de la connexion d'un joueur unique, nous devons filtré pour retirer les doublons.

In [33]:
dataplot = wowah_data.groupby('dates')['char'].nunique().to_frame().reset_index()

fig = px.line(dataplot, x="dates", y="char" ,hover_data=['dates', 'char'],
              labels={'char':'Unique Player', 'charclass' : 'Class', 'lvl_inter2' : 'lever', 'pers' : 'Percentage', 'dates' : 'Date'}, height=400).update_traces(mode='lines+markers')

fig = px.line(dataplot, x="dates", y="char",
             hover_data=['char'], 
             labels={'char':' Population'}, height=400)

fig.add_shape(
        dict(
            type="line",
            x0='2008-11-18',
            y0=0,
            x1='2008-11-18',
            y1=4000,
            line=dict(
                color="Black",
                width=2
            )))
fig.add_trace(go.Scatter(
    x=['2008-9-30', '2008-12-30'],
    y=[3500, 3500],
    text=["Burning Crusade",
          "WOTLK",],
    mode="text",
))

fig.show()

Sur ce graphe on remarque le **comportement des joueurs**, nous pouvons clairement voir qu'en fonction du jour, le nombre de snapshot change.

Le **Samedi/Dimanche** les joueurs ont plus le temps de jouer et il y a donc plus de connexion au contraire du **Mardi/Jeudi** où les gens ont tendance à moins jouer.

Cependant on remarque des **chutes** qui peuvent être lié à un incident (serveur).

Incident : 27 Mars => https://worldofwarcraft.judgehype.com/news/des-problemes-pour-le-corps-de-bataille-cataclysme-113893/

Plus gros pic : 8 Octobre => Blizzcon 2008 (Arena Tournament)

In [38]:
dataplot = wowah_data.copy()

dataplot['week'] = dataplot.timestamp.dt.day_name()
dataplot = dataplot.groupby(['dates','week'])['char'].nunique().to_frame().reset_index()

dataplot = wowah_data.copy()
dataplot['week'] = dataplot.timestamp.dt.day_name()
dataplot = dataplot.groupby(['week'])['char'].nunique().to_frame().reset_index()
dataplot = dataplot.sort_values(by = 'char')

fig = px.bar(dataplot, x='week', y='char', color = 'week',
             hover_data=['week', 'char'],  color_discrete_map =  dict_color,
             labels={'char':'Active Players', 'charclass': 'Character Class', 'rece' : 'Character Race'}, height=500)

fig.show()

del dataplot

Le mercredi est assez populaire puisqu'il y a la maintenance hebdomadaire avec la réinitialisation de beaucoup de mécanique qui permet la progression de votre personnage.

### Comparaison entre la répartition des classes avant et après l'extension.
 <a id="2"></a>

In [34]:
Dataplot = pd.DataFrame(wowah_data.groupby('charclass').count()['char']).reset_index().sort_values(by = 'char')
Dataplot['pers'] = Dataplot['char'].div(108267.34).round(1).astype(str) + '%'

datatmp1 = wowah_data[wowah_data['timestamp'] < '2008-11-18'].groupby([pd.Grouper(key='timestamp', freq='30d'), 'charclass']).count().reindex().groupby('charclass').mean()
datatmp1['pers'] = datatmp1['char'].div(datatmp1.char.sum()/100).round(1).astype(str) + '%'

datatmp2 = wowah_data[wowah_data['timestamp'] >= '2008-11-18'].groupby([pd.Grouper(key='timestamp', freq='30d'), 'charclass']).count().reindex().groupby('charclass').mean()
datatmp2['pers'] = datatmp2['char'].div(datatmp2.char.sum()/100).round(1).astype(str) + '%'

datatmp1['ext'] = 'BC'
datatmp2['ext'] = 'WOTLK'

Dataplot = pd.concat([datatmp1,datatmp2]).reset_index()

fig = px.bar(Dataplot, x='charclass', y='char', color = 'charclass',hover_data=['charclass', 'char'], color_discrete_map =  dict_color, facet_row='ext',
             labels={'char':'Popularity', 'charclass' : 'Class', 'ext' : 'Extention', 'pers' : '%'}, height=400, text = 'pers')

fig.show()

Les classes les plus jouer sont bien souvent les plus performantes et qui apporte le plus de polyvalence et souple dans sa manière de jouer.

Chaque classe peut avoir un ou plusieurs rôle (infligez des dégâts, subir des dégâts ou encore soigner).

Les classes populaire sont Chasseur (Hunter), Mage, Guerrier (Warrior) et Démoniste (Warlock). 

Les autres classes sont plus niches et offre moins de possibilité.

L'équilibrage change aussi la popularité d'une classe (cependant nous avons pas cette donnée...).

La nouvelle classe 'Death Knight' était très populaire au début de la nouvelle extension, le temps que l'ensemble des joueurs essaye la classe, grand nombre seraient resté sur leur personnage d'originel.

### Evolution de la répartition des classes entre deux extensions.
 <a id="2"></a>

In [14]:
Dataplot = pd.DataFrame(wowah_data.groupby(['dates', 'charclass']).count()['char']).reset_index().sort_values(by = 'dates')
Dataplot2 = pd.DataFrame(wowah_data.groupby(['dates']).count()['char']).reset_index().sort_values(by = 'dates')
Dataplot = Dataplot.merge(Dataplot2, left_on = 'dates', right_on = 'dates')
Dataplot['pers'] = Dataplot['char_x'].div(Dataplot.char_y.values /100).round(1)

fig = px.line(Dataplot, x="dates", y="pers", color='charclass',
             hover_data=['charclass', 'pers'], 
             color_discrete_map =  dict_color,
             labels={'char':'Class Population', 'charclass' : 'Class', 'pers' : 'Class Population Percentage'}, height=400)

fig.add_shape(
        dict(
            type="line",
            x0='2008-11-18',
            y0=0,
            x1='2008-11-18',
            y1=40,
            line=dict(
                color="Black",
                width=2
            )))
fig.add_trace(go.Scatter(
    x=['2008-9-30', '2008-12-30'],
    y=[30, 30],
    text=["Burning Crusade",
          "WOTLK",],
    mode="text",
))

Dataplot = pd.DataFrame(wowah_data.groupby(['dates', 'charclass']).count()['char']).reset_index().sort_values(by = 'dates')

fig.show()

Pendant la période de la première extension Burning Crusade, la population de classe est rester plus ou moins la même. Le chasseur est le plus jouer au contraire du chaman.

Les variances des pourcentages de population sont probablement dû à des mise à jour destiné à l'équilibrage des classes (Pré-mise à jour de l'extension).

La nouvelle classe à WOTLK reçoi un énorme pic qui se réduit avec le temps (des joueurs ayant essayé la classe sans suite).



### Evolution de la répartition des joueurs dans les différentes régions.
 <a id="2"></a>

In [36]:
Dataplot = pd.DataFrame(wowah_data.groupby(['dates', 'zone']).count()['char']).reset_index().sort_values(by = ['dates', 'char'])
Dataplot = Dataplot.groupby(['dates']).tail(2).reset_index().sort_values(by = ['dates', 'char'])

fig = make_subplots(rows=3, cols=1)

fig = px.scatter(Dataplot, x="dates", y="char", color='zone',
             hover_data=['zone', 'char'], 
             labels={'char':'Population', 'dates' : 'Date'}, height=400)

fig.add_shape(
        dict(
            type="line",
            x0='2008-11-18',
            y0=0,
            x1='2008-11-18',
            y1=7000,
            line=dict(
                color="Black",
                width=2
            )))
fig.add_trace(go.Scatter(
    x=['2008-9-30', '2008-12-30'],
    y=[6000, 5000],
    text=["Burning Crusade",
          "WOTLK",],
    mode="text",
))


Dataplot = pd.DataFrame(wowah_data.groupby(['First_day_of_the_week', 'zone']).count()).reset_index()[['char', 'First_day_of_the_week','zone']].sort_values(by = ['First_day_of_the_week', 'char'])
Dataplot.First_day_of_the_week = pd.to_datetime(Dataplot.First_day_of_the_week).dt.strftime('%m/%d/%Y')
fig.show()



Avant la nouvelle extension : On remarque que les joueurs se retrouve essentiellement dans les différentes capitales (Orgrimmar, Shattrath) ou bien des zones de combat comme Karazhan/Quel'Danas.

Après la sortie de la nouvelle extension : On remarque que principalement les zones sont des nouvelles régions de l'extension qui d'une partie permettent de gagner en niveau (Bassin Arathi, Dragonblight...) ou la nouvelle capitale Dalaran.

### Conclusion.
 <a id="2"></a>

Pour comprendre plus en détail le comportement des joueurs lors d'une nouvelle extensions (choix zone, classe, race, connexion des joueurs...) il manque un trop grand nombre de donnée (mise à jour, équilibrage, répartition du serveur...).


